In [2]:
import pandas as pd
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init
import itertools
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
import csv
import time

In [3]:
#methanol = component(name='methanol', Tc=512.5, Pc=80.84, Zc=0.222, Vc=117, GC={'CH3OH':1})
#ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168, GC={'CH3':1, 'CH2':1, 'OH':1})
#_2propanol = component(name='2-propanol', Tc=508.3, Pc=47.65, Zc=0.25, Vc=222, GC={'CH3':2, 'CH2':1, 'OH':1})
#acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213, GC={'CH3CO':1, 'CH3':1})
#mibk = component(name='mibk', Tc=574.6, Pc=32.7, Zc=0.256, Vc=317, GC={'CH3CO':1, 'CH3':2, 'CH2':1, 'CH':1})
#ethac = component(name='ethyl-acetate', Tc=523.3, Pc=38.8, Zc=0.255, Vc=286, GC={'CH3COO':1, 'CH3':1, 'CH2':1})
#chloroform = component(name='chloroform', Tc=536.4, Pc=54.72, Zc=0.293, Vc=239, GC={'CHCL3':1})
#toluene = component(name='toluene', Tc=591.75, Pc=41.08, Zc=0.264, Vc=316, GC={'ACH':5, 'ACCH3':1})
#water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,GC={'H2O':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1})
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,
                  GC={'H2O':1}) # water

npentane = component(name='n-pentane', Tc=469.7, Pc=33.7, Zc=0.27, Vc=313,
                     GC={'CH3':2, 'CH2':3}) # n-pentane

nheptane = component(name='n-heptane', Tc=540.2, Pc=27.4, Zc=0.261, Vc=428,
                     GC={'CH3':2, 'CH2':5}) # n-heptane

ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168,
                    GC={'CH3':1, 'CH2':1, 'OH':1}) # ethanol

_1propanol = component(name='1-propanol', Tc=536.8, Pc=51.69, Zc=0.254, Vc=219,
                       GC={'CH3':1, 'CH2':2, 'OH':1}) # 1-propanol

_1butanol = component(name='1-butanol', Tc=563.1, Pc=44.14, Zc=0.258, Vc=273,
                      GC={'CH3':1, 'CH2':3, 'OH':1})

_1pentanol = component(name='1-pentanol', Tc=588.1, Pc=38.97, Zc=0.258, Vc=326,
                       GC={'CH3':1, 'CH2':4, 'OH':1})

acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213,
                    GC={'CH3CO':1, 'CH3':1})

In [4]:
#component_list = [methanol,ethanol,_2propanol,acetone,mibk,ethac,chloroform,toluene,water]
component_list = [water,npentane,nheptane,ethanol,_1propanol,_1butanol,_1pentanol,acetone]
#component_list= [water,npentane,nheptane,ethanol,_1propanol,_1butanol,_1pentanol,acetone,_12propanediol,acetic_acid,
#                cyclohexane, isobutanol, isopropanol, mibk, mek, hexane, isohexane, _2pentanol, _2pentanone, mchexane,
#                mcpentane, ethac, ibutac, ipropac, methac, nbutac, propac, tbutanol, tAmOH, _224tmpentane, tbutac, mtbe,
#                _12dimethoxyethane, diethyl_ether, _14dioxane, _2methylTHF, THF, o_xylene, toluene, anisole, cyclohexanone,
#                _3pentanone, chloroform, methanol, formic_acid, bromobutane, acetonitrile, chlorobenzene, methylene_chloride,
#                triethamine, trifluoroaceticacid]
comp_no = len(component_list)
print(comp_no)

8


In [5]:
qs = {
    "methanol" : 1.432,
    "ethanol" : 2.588,
    "2-propanol" : 3.436,
    "acetone" : 2.336,
    "mibk" : 3.952,
    "ethyl-acetate" : 3.116,
    "chloroform" : 2.41,
    "toluene" : 2.968,
    "water" : 1.4,
    "hexane" : 3.856,
    "heptane" : 4.396,
    "propylacetate" : 3.656,
    "butylacetate" : 4.196,
    "o-xylene" : 3.536,
    "butanone" : 2.876,
    "butanol" : 3.668,
    "12propanediol" : 4.016,
    "13propanediol" : 4.02,
    "n-pentane" : 3.316,
    "n-heptane" : 4.396,
    "1-propanol" : 3.128,
    "1-pentanol" : 4.208,
    "1-butanol" : 3.668,
    "acetic-acid" : 2.072,
    "cyclohexane" : 3.24,
    "iso-butanol" : 3.664,
    "iso-propanol" : 3.124,
    "mek" : 2.876,
    "2-methylpentane" : 3.852,
    "2-pentanol" : 4.204,
    "2-pentanone" : 3.416,
    "methylcyclohexane" : 3.776,
    "methylcyclopentane" : 3.236,
    "isobutyl-acetate" : 4.192,
    "isopropyl-acetate" : 3.964,
    "methyl-acetate" : 2.576,
#    "n-butyl-acetate" : 3.116,
    "propyl-acetate" : 3.656,
    "t-butanol" : 3.744,
    "t-AmOH" : 4.284,
    "2,2,2-trimethylpentane" : 5.008,
    "t-butyl-acetate" : 4.272,
    "mtbe" : 1.936,
    "1,2-dimethoxyethane" : 3.256,
    "diethyl ether" : 3.016,
    "1,4-dioxane" : 3.28,
    "2-methyl THF" : 2.716,
    "THF" : 2.72,
    "anisole" : 3.208,
    "cyclohexanone" : 3.34,
    "3-pentanone" : 3.416,
    "formic acid" : 1.532,
    "1-bromobutane" : 2.22,
    "acetontrile" : 1.724,
    "chlorobenzene" : 2.844,
    "methylene chloride" : 1.988,
    "triethylamine" : 4.256,
    "trifluoro-acetic acid" : 2.604
}

rs = {
    "methanol" : 1.431,
    "ethanol" : 2.576,
    "2-propanol" : 3.477,
    "acetone" : 2.574,
    "mibk" : 3.952,
    "ethyl-acetate" : 3.479,
    "chloroform" : 2.87,
    "toluene" : 3.923,
    "water" : 0.92,
    "hexane" : 4.5,
    "heptane" : 5.174,
    "propylacetate" : 4.153,
    "butylacetate" : 4.827,
    "o-xylene" : 4.658,
    "butanone" : 3.248,
    "butanol" : 3.924,
    "12propanediol" : 4.022,
    "13propanediol" : 4.023,
    "n-pentane" : 3.825,
    "n-heptane" : 5.174,
    "1-propanol" : 3.25,
    "1-pentanol" : 4.599,
    "1-butanol" : 3.924,
    "acetic-acid" : 2.202,
    "cyclohexane" : 4.046,
    "iso-butanol" : 3.923,
    "iso-propanol" : 3.249,
    "mek" : 2.876,
    "2-methylpentane" : 4.499,
    "2-pentanol" : 4.598,
    "2-pentanone" : 3.922,
    "methylcyclohexane" : 4.72,
    "methylcyclopentane" : 4.046,
    "isobutyl-acetate" : 4.827,
    "isopropyl-acetate" : 4.83,
    "methyl-acetate" : 2.804,
    "n-butyl-acetate" : 3.479,
    "propyl-acetate" : 4.153,
    "t-butanol" : 3.923,
    "t-AmOH" : 4.597,
    "2,2,2-trimethylpentane" : 5.846,
    "t-butyl-acetate" : 4.826,
    "mtbe" : 2.266,
    "1,2-dimethoxyethane" : 3.639,
    "diethyl ether" : 3.614,
    "1,4-dioxane" : 3.185,
    "2-methyl THF" : 2.941,
    "THF" : 2.942,
    "anisole" : 4.167,
    "cyclohexanone" : 4.143,
    "3-pentanone" : 3.922,
    "formic acid" : 1.528,
    "1-bromobutane" : 2.525,
    "acetontrile" : 1.87,
    "chlorobenzene" : 3.813,
    "methylene chloride" : 2.256,
    "triethylamine" : 5.012,
    "trifluoro-acetic acid" : 2.707
}

In [6]:
inv_qs = dict(map(reversed, qs.items()))

In [7]:
component_map = {"ethanol" : ethanol, "1-propanol" : _1propanol, "acetone" : acetone, "1-butanol" : _1butanol, 
                 "1-pentanol" : _1pentanol, "n-pentane" : npentane, "n-heptane" : nheptane, "water" : water}

In [15]:
st = time.time() #start timer
nmin = 3
P = 1.01325
master_list = []
with open('tpd2_data_no_api_wT.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        comp1 = component_map[inv_qs[float(row[1])]]
        comp2 = component_map[inv_qs[float(row[2])]]
        mix_func = mixture(comp1,comp2)
        mix_func += ibuprofen
        mix_func.original_unifac()
        eos = virialgamma(mix_func, virialmodel='ideal_gas', actmodel='original_unifac')
        z1 = float(row[5])
        z2 = float(row[6])
        z3 = 1 - z1 - z2
        z = np.array([z1,z2,z3])
        T = float(row[7])
        tpd_m = tpd_minimas(nmin, z, T, P, eos, stateW='L', stateZ='L')
        tpd_values = tpd_m[-1] #outputs nmin TPD values 
        min_tpd_value = min(tpd_values) #takes the minimum TPD value
        if min_tpd_value < 0:
            ismisc = 0
        else:
            ismisc = 1
        min_tpd_value_index = np.where(tpd_values == min_tpd_value) #indexes the minimum TPD value, so its corresponding w can be displayed      
        min_tpd_value_index = min_tpd_value_index[0][0] #for repeat solutions, only takes first in the list
        tpd_m_w = tpd_m[0][min_tpd_value_index]
        tpd_m_w = tpd_m_w.tolist() #converts type from array to list
        
        master_list.append([T,mix_func.names[0],mix_func.names[1],z1,z2,min_tpd_value,ismisc])

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds') #execution time

Execution time: 1923.302523612976 seconds


In [16]:
s1 = [None]*len(master_list)
s2 = [None]*len(master_list)
q1 = [None]*len(master_list)
q2 = [None]*len(master_list)
r1 = [None]*len(master_list)
r2 = [None]*len(master_list)
x1 = [None]*len(master_list)
x2 = [None]*len(master_list)
T = [None]*len(master_list)
ismisc = [None]*len(master_list)

In [17]:
for i in range(len(master_list)):
    q1[i]=qs[master_list[i][1]]
    q2[i]=qs[master_list[i][2]]
    r1[i]=rs[master_list[i][1]]
    r2[i]=rs[master_list[i][2]]
    x1[i]=master_list[i][3]
    x2[i]=master_list[i][4]
    T[i]=master_list[i][0]
    ismisc[i]=master_list[i][6]

In [18]:
df = pd.DataFrame({"q1":q1,"q2":q2,"r1":r1,"r2":r2,"x1":x1,"x2":x2,"T":T,"ismisc":ismisc})
df.to_csv('tpd2_data_wT.csv')

In [14]:
from tabulate import tabulate
print(tabulate(master_list, headers=["Temp", "Component 1","Component 2", "z1", "z2", 'TPD min', 'ismisc'],tablefmt="presto"))

    Temp | Component 1   | Component 2   |          z1 |          z2 |        TPD min |   ismisc
---------+---------------+---------------+-------------+-------------+----------------+----------
 302.264 | water         | n-pentane     | 0.396605    | 0.43427     |   -1.3729      |        0
 316.256 | water         | n-pentane     | 0.282085    | 0.312005    |   -0.901941    |        0
 298.715 | water         | n-pentane     | 0.0109012   | 0.769412    |    7.01661e-14 |        1
 306.549 | water         | n-pentane     | 0.411134    | 0.0613471   |   -0.825275    |        0
 310.52  | water         | n-pentane     | 0.323066    | 0.371168    |   -1.15433     |        0
 293.157 | water         | n-pentane     | 0.219572    | 0.277086    |   -0.583452    |        0
 313.898 | water         | n-pentane     | 0.128289    | 0.404704    |   -0.285845    |        0
 303.084 | water         | n-pentane     | 0.679151    | 0.259121    |   -0.906108    |        0
 296.225 | water         | n-